In [ ]:
cd ..

/content/drive/My Drive/seq2seq_coref


In [ ]:

# download entity_recognizer_dl pipeline from Spark-NLP S3 repository
pipeline = PretrainedPipeline('recognize_entities_dl', lang='en')

recognize_entities_dl download started this may take some time.
Approx size to download 159 MB
[OK!]


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/seq2seq_coref

/content/drive/My Drive/seq2seq_coref


In [ ]:
cd reference-coreference-scorers/

/content/drive/My Drive/seq2seq_coref/reference-coreference-scorers


In [ ]:
!perl scorer.pl all /content/drive/My\ Drive/seq2seq_coref/more_data/pred_int.conll  /content/drive/My\ Drive/seq2seq_coref/more_data/pred_out.conll

Streaming output truncated to the last 5000 lines.
Entity 0: (2,2)
Entity 1: (29,29) (63,63)
Entity 2: (39,39)
Entity 3: (68,69) (78,79)
(bc/cctv/00/cctv_0000); part 004:
Total key mentions: 5
Total response mentions: 6
Strictly correct identified mentions: 3
Partially correct identified mentions: 0
No identified: 2
Invented: 3
Recall: (0 / 1) 0%	Precision: (0 / 2) 0%	F1: 0%
--------------------------------------------------------------------------
====> (bc/cnn/00/cnn_0000); part 007:
File (bc/cnn/00/cnn_0000); part 007:
Entity 0: (2,2) (32,32) (54,54)
Entity 1: (5,29)
Entity 2: (25,29) (50,51)
Entity 3: (56,64) (68,68) (73,73) (79,79) (109,109) (115,115)
Entity 4: (100,102)
Entity 5: (109,110) (115,116)
Entity 6: (144,145)
====> (bc/cnn/00/cnn_0000); part 007:
File (bc/cnn/00/cnn_0000); part 007:
Entity 0: (58,59)
Entity 1: (63,64) (68,68) (73,73)
Entity 2: (100,102)
(bc/cnn/00/cnn_0000); part 007:
Total key mentions: 16
Total response mentions: 5
Strictly correct identified mentions

In [3]:

#read files
#text = pd.read_csv('train.english.v4_gold_conll', sep=' ')
text = open('test.english.v4_gold_conll', mode='r')

In [4]:
text_lines = text.readlines()

In [5]:
end_count = sum([1 for line in text_lines if line.startswith("#end")])
print(end_count)

348


In [18]:
text_lines1 =  text_lines[:30515]

In [19]:
text_lines1

['#begin document (bc/cctv/00/cctv_0005); part 000\n',
 'bc/cctv/00/cctv_0005   0    0            --      :  (TOP(S*       -    -   -   speaker#1   *      *             *   -\n',
 'bc/cctv/00/cctv_0005   0    1      basically    RB   (ADVP*)      -    -   -   speaker#1   *      *    (ARGM-ADV*)  -\n',
 'bc/cctv/00/cctv_0005   0    2              ,     ,        *       -    -   -   speaker#1   *      *             *   -\n',
 'bc/cctv/00/cctv_0005   0    3             it   PRP     (NP*)      -    -   -   speaker#1   *      *             *   -\n',
 'bc/cctv/00/cctv_0005   0    4            was   VBD     (VP*       be  03   -   speaker#1   *    (V*)            *   -\n',
 'bc/cctv/00/cctv_0005   0    5    unanimously    RB   (ADVP*)      -    -   -   speaker#1   *      *    (ARGM-MNR*)  -\n',
 'bc/cctv/00/cctv_0005   0    6         agreed   VBN     (VP*    agree  01   1   speaker#1   *      *           (V*)  -\n',
 'bc/cctv/00/cctv_0005   0    7           upon    IN     (PP*)      -    -   

In [20]:
documents = []
sentences = []
l_words = []

for line in text_lines1:
  if line.startswith('#end'):
    documents.append(sentences)
    sentences=[]
  # line.replace('#end', '\n')
    # pass
  elif line.startswith("#begin"):
    continue
  else:
    if line == '\n':
      sentence = ' '.join(l_words)
      sentences.append(sentence)
      l_words = []
      continue
    else:
      #print(1,line)
      id = line.split()[1]
      #for i in id:
      word = line.split()[3]
      l_words.append(word)
      # l_words.append('\n')
      #i+=1

In [21]:

print("read" , len(documents), "documents composed of ", sum([len(sentences) for sentences in documents]), "sentences")

read 50 documents composed of  1798 sentences


In [22]:
documents

[['-- basically , it was unanimously agreed upon by the various relevant parties .',
  'To express its determination , the Chinese securities regulatory department compares this stock reform to a die that has been cast .',
  'It takes time to prove whether the stock reform can really meet expectations , and whether any deviations that arise during the stock reform can be promptly corrected .',
  'Dear viewers , the China News program will end here .',
  'This is Xu Li .',
  'Thank you everyone for watching .',
  'Coming up is the Focus Today program hosted by Wang Shilin .',
  'Good-bye , dear viewers .'],
 ['Hello , dear viewers .',
  'Welcome to Focus Today .',
  "Today , let 's turn our attention to a road cave - in accident that happened in Beijing over the holiday .",
  'Before dawn on January 3 , a sewage pipe leakage accident occurred at the main and side roads of Jingguang Bridge , East Third Ring Road , Beijing Municipality , resulting in the road caving in .',
  'Relevant dep

In [ ]:
# for d in documents:
#   print(d)

In [23]:
file = open('test_data/test.source', mode='a+')
for d in documents:
  file.write(str(d)+"\n")
  #file.write("#end\n")
file.close()

In [ ]:
cd ..

/content/drive/My Drive/seq2seq_coref


In [ ]:
!wc -l * 

    50 dev.source
   100 train.source
   150 total


In [24]:


with open('test_data/test.source', 'r+') as f:
  log = f.readlines()


In [26]:
log

["['-- basically , it was unanimously agreed upon by the various relevant parties .', 'To express its determination , the Chinese securities regulatory department compares this stock reform to a die that has been cast .', 'It takes time to prove whether the stock reform can really meet expectations , and whether any deviations that arise during the stock reform can be promptly corrected .', 'Dear viewers , the China News program will end here .', 'This is Xu Li .', 'Thank you everyone for watching .', 'Coming up is the Focus Today program hosted by Wang Shilin .', 'Good-bye , dear viewers .']\n",
 '[\'Hello , dear viewers .\', \'Welcome to Focus Today .\', "Today , let \'s turn our attention to a road cave - in accident that happened in Beijing over the holiday .", \'Before dawn on January 3 , a sewage pipe leakage accident occurred at the main and side roads of Jingguang Bridge , East Third Ring Road , Beijing Municipality , resulting in the road caving in .\', \'Relevant departments 

In [27]:
for l in log:
  l = l.replace("\\", "")
  print(l)

['-- basically , it was unanimously agreed upon by the various relevant parties .', 'To express its determination , the Chinese securities regulatory department compares this stock reform to a die that has been cast .', 'It takes time to prove whether the stock reform can really meet expectations , and whether any deviations that arise during the stock reform can be promptly corrected .', 'Dear viewers , the China News program will end here .', 'This is Xu Li .', 'Thank you everyone for watching .', 'Coming up is the Focus Today program hosted by Wang Shilin .', 'Good-bye , dear viewers .']

['Hello , dear viewers .', 'Welcome to Focus Today .', "Today , let 's turn our attention to a road cave - in accident that happened in Beijing over the holiday .", 'Before dawn on January 3 , a sewage pipe leakage accident occurred at the main and side roads of Jingguang Bridge , East Third Ring Road , Beijing Municipality , resulting in the road caving in .', 'Relevant departments from Beijing Mu

In [ ]:
!pip install -U spacy

     |████████████████████████████████| 10.0MB 5.4MB/s 
     |████████████████████████████████| 2.1MB 46.2MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
# import spacy
# import neuralcoref

In [ ]:
!python -m spacy download en

     |████████████████████████████████| 12.1MB 2.6MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp36-none-any.whl size=12047109 sha256=2b6dd6cee332617a5ba6aa9099aef0afac24dd5ae29b186a6bbc70e5bc55cbce
  Stored in directory: /tmp/pip-ephem-wheel-cache-m_osvuig/wheels/2b/3f/41/f0b92863355c3ba34bb32b37d8a0c662959da0058202094f46
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
#!git clone https://github.com/huggingface/neuralcoref.git

In [ ]:
cd neuralcoref/

/content/drive/My Drive/seq2seq_coref/neuralcoref


In [ ]:
pip install -r requirements.txt

In [ ]:
!pip install -e .

Obtaining file:///content/drive/My%20Drive/seq2seq_coref/neuralcoref
     |████████████████████████████████| 133kB 5.1MB/s 
     |████████████████████████████████| 71kB 4.5MB/s 
     |████████████████████████████████| 6.8MB 5.8MB/s 
ERROR: botocore 1.19.21 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
  Running setup.py develop for neuralcoref


In [ ]:
import spacy
nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)
#neuralcoref.add_to_pipe(nlp, greedyness=0.75)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:00<00:00, 67585826.07B/s]


In [ ]:
len(log._.coref_clusters)

AttributeError: ignored

In [ ]:
# entities = doc1._.coref_clusters
# all_mentions = [(mention,i+1) for i,e in enumerate(entities) for mention in e]
# #print(all_mentions)
# sorted_mentions = sorted(all_mentions,key = lambda x:x[0].start_char)
# #print(sorted_mentions)
# out_string1 = str(doc1)
# out = []
# prev_end=1
# for mention, i in sorted_mentions:
#   prefix = out_string1[prev_end:mention.start_char]
#   #print(prefix)
#   prev_end = mention.end_char
#   out.append(prefix)
#   out.append("{"+mention.string.strip()+"}"+" ["+str(i)+"]")
#   #print(out)
# #
  
# out_string1 = "".join(out)
#       #out_string.replace(mention, "{"+mention+"}"+" ["+i+"]")
# print(out_string1)

In [ ]:
def is_overlap(mentions):
  overlap=0
  for i, (mention1,_) in enumerate(mentions):
    for j,(mention2,_) in enumerate(mentions[i+1:]):
      if (mention1.start>mention2.start and mention1.end<mention2.end) or (mention2.start>mention1.start and mention2.end<mention1.end):
        overlap+=1
  return overlap


In [ ]:

for doc2 in log:
  #doc_str = "\n".join(doc2)
  print(doc2)
  #document1 = nlp(doc_str)
#print(document1)

What kind of memory ? We respectfully invite you to watch a special edition of Across China . WW II Landmarks on the Great Earth of China : Eternal Memories of Taihang Mountain' Standing tall on Taihang Mountain is the Monument to the Hundred Regiments Offensive . It is composed of a primary stele , secondary steles , a huge round sculpture and beacon tower , and the Great Wall , among other things . A primary stele , three secondary steles , and two inscribed steles . The Hundred Regiments Offensive was the campaign of the largest scale launched by the Eighth Route Army during the War of Resistance against Japan . This campaign broke through the Japanese army 's blockade to reach base areas behind enemy lines , stirring up anti-Japanese spirit throughout the nation and influencing the situation of the anti-fascist war of the people worldwide . This is Zhuanbi Village , Wuxiang County of Shanxi Province , where the Eighth Route Army was headquartered back then . On a wall outside the h

In [ ]:
documents_coref = []
searched_docs = []
for document in log:
  document_str = document
 # print("number of lines before",len(document))
  document1 = nlp(document_str)
 # print("number of lines after",len(list(document.sents)))
  entities = document1._.coref_clusters
  #print("found ",len(document._.coref_clusters),"entities")
  all_mentions = [(mention,i+1) for i,e in enumerate(entities) for mention in e]
  # for mention,_ in all_mentions:
  #   if "\n" in mention.string:
  #     print("###########",mention)
  # if is_overlap(all_mentions):
  #   continue
  #searched_docs.append(document)
  sorted_mentions = sorted(all_mentions,key = lambda x:x[0].start_char)
  #print(sorted_mentions)
  out_string = document_str
  out = []
  prev_end=0
  for mention, i in sorted_mentions:
    prefix = out_string[prev_end:mention.start_char]
    #print(prefix)
    prev_end = mention.end_char + 1
    out.append(prefix)
    out.append("{ "+mention.string+" } "+" [ "+str(i)+" ] " )
    #print(out)
  #
    
  out_string = "".join(out)
        #out_string.replace(mention, "{"+mention+"}"+" ["+i+"]")
  # print(out_string)
  documents_coref.append(out_string)

In [ ]:

documents_coref


["What kind of memory ? We respectfully invite you to watch a special edition of Across { China  }  [ 1 ] . WW II Landmarks on the Great Earth of { China  }  [ 1 ] : { Eternal Memories of Taihang Mountain' Standing tall on Taihang Mountain  }  [ 2 ] is the Monument to { the Hundred Regiments Offensive  }  [ 3 ] . { It  }  [ 2 ] is composed of a primary stele , secondary steles , a huge round sculpture and beacon tower , and the Great Wall , among other things . A primary stele , three secondary steles , and two inscribed steles . { The Hundred Regiments Offensive  }  [ 3 ] was { the campaign of the largest scale launched by the Eighth Route Army during the War of Resistance against Japan  }  [ 4 ] { the Eighth Route Army  }  [ 5 ] during the War of Resistance against Japan . { This campaign  }  [ 4 ] broke through { the Japanese army  }  [ 6 ] 's blockade to reach base areas behind enemy lines , stirring up anti-Japanese spirit throughout { the nation  }  [ 1 ] and influencing the situ

In [ ]:
documents_coref[0].replace("\\", " ")

'[\'In the summer of 2005 , a picture that people have long been looking forward to started emerging with frequency in various major { Hong Kong  }  [ 1 ] media .\', \'With their unique charm , these well - known cartoon images once again caused { Hong Kong  }  [ 1 ] to be a focus of worldwide attention .\', "The world \'s fifth { Disney  }  [ 2 ] park will soon open to the public here .", \'The most important thing about { Disney  }  [ 2 ] is that { it  }  [ 2 ] is a global brand .\', \'Well , for several years , although it was still under construction and , er , not yet open , it can be said that many people have viewed { Hong Kong  }  [ 1 ] with new respect .\', \'Then welcome to the official writing ceremony of { Hong Kong Disneyland  }  [ 3 ] .\', \'The construction of { Hong Kong Disneyland  }  [ 3 ] began two years ago , in 2003 .\', \'In January of that year , the Hong Kong government turned over to Disney Corporation 200 hectares of land at the foot of { Lantau Island  }  [ 4

In [ ]:
print("read" , len(documents_coref), "documents composed of ", sum([len(sentences.split('\n')) for sentences in documents_coref]), "sentences")

read 100 documents composed of  100 sentences


In [ ]:
print("number of sentences in the read documents", sum([len(list(document.sents)) for document in searched_docs]))

number of sentences in the read documents 0


In [ ]:
file = open('../more_data/train.target', mode='w+')
for dd in documents_coref:
  file.write(dd+"\n")
file.close()

In [ ]:
cd more_data/

/content/drive/My Drive/seq2seq_coref/more_data


In [ ]:
!wc -l * 

wc: bin: Is a directory
      0 bin
     50 dev.source
     50 dev.target
    100 train.source
    100 train.target
    300 total


In [ ]:

# import spacy
# from spacy import displacy
# from collections import Counter
# import en_core_web_sm
# nlp = en_core_web_sm.load()

In [ ]:
#displacy.render(nlp(str(article)), style='dep', jupyter = True, options = {'distance': 43})

In [ ]:
#NeuralCoref is an open source python packgage integrated in SpaCy’s NLP pipeline